In [2]:
import numpy as np
import pennylane as qml

In [ ]:
def werner_state(start_wire,dimension,psi):
    qml.RY(psi,start_wire)
    qml.Hadamard(start_wire+dimension+1)
    for i in range (start_wire+dimension+2,start_wire+2*dimension+1):
        qml.CNOT([start_wire+dimension+1,i])
    for i in range (start_wire+dimension+2,start_wire+2*dimension+1):
        qml.MultiControlledX([start_wire,start_wire+dimension+1],i)
    qml.ctrl(qml.Hadamard,start_wire)(start_wire+dimension+1)
    for i in range (start_wire+1,start_wire+dimension+1):
        qml.ctrl(qml.Hadamard,start_wire)(i)
    for i in range (start_wire+dimension+1,start_wire+2*dimension+1):
        qml.MultiControlledX([start_wire,i-dimension],i)


In [4]:
dev_swap = qml.device("lightning.qubit", wires=17)
@qml.qnode(dev_swap)
def SWAP_CE_werner_state(dimension,phi):
    for i in range (dimension):
        qml.Hadamard(i)
    werner_state(dimension,dimension,phi)
    werner_state(dimension+1+2*dimension,dimension,phi)
    for i in range (dimension):
        qml.CSWAP([i,i+2*dimension+1,i+4*dimension+2])
    for i in range (dimension):
        qml.Hadamard(i)
    return (qml.probs([i for i in range (dimension)]))

In [5]:
drawer = qml.draw(SWAP_CE_werner_state)
print(drawer(2,0))

 0: ──H─────────────────────────────╭●─────H───────┤ ╭Probs
 1: ──H─────────────────────────────│─────╭●─────H─┤ ╰Probs
 2: ──RY(0.00)────╭●─╭●─╭●─╭●─╭●─╭●─│─────│────────┤       
 3: ──────────────│──│──╰H─│──├●─│──│─────│────────┤       
 4: ──────────────│──│─────╰H─│──├●─│─────│────────┤       
 5: ──H────────╭●─├●─╰H───────╰X─│──├SWAP─│────────┤       
 6: ───────────╰X─╰X─────────────╰X─│─────├SWAP────┤       
 7: ──RY(0.00)────╭●─╭●─╭●─╭●─╭●─╭●─│─────│────────┤       
 8: ──────────────│──│──╰H─│──├●─│──│─────│────────┤       
 9: ──────────────│──│─────╰H─│──├●─│─────│────────┤       
10: ──H────────╭●─├●─╰H───────╰X─│──╰SWAP─│────────┤       
11: ───────────╰X─╰X─────────────╰X───────╰SWAP────┤       


/home/ruibinx/.local/lib/python3.9/site-packages/pennylane/ops/qubit/non_parametric_ops.py:2095: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(


In [6]:
dev_density = qml.device("lightning.qubit", wires=17)
@qml.qnode(dev_density)
def density_wener_state(dimension,psi):
    werner_state(0,dimension,psi)
    return qml.density_matrix([i for i in range (2*dimension-dimension+1+0,2*dimension+1+0)])

In [ ]:
Phi = np.linspace(0, 2*np.pi, 50)
def classical(n,i):
    p=(np.cos(i/2))**2
    w=density_wener_state(n,i)
    Pu=np.trace(np.dot(w,w))
    n1=(1-1/2**(n-1))*Pu
    n2_1=(2**n-2)*p**2/2
    n2=n2_1+(1-p**2)*(3**n-2**n-1)/2**n
    n3=n2/2**n
    return(n1-n3)
def CE(n,i):
    w=density_wener_state(n,i)
    P=np.trace(np.dot(w,w))
    sum_=SWAP_CE_werner_state(n,i)[0]
    return(1/2**n+(1-1/2**n)*P-sum_)

/home/ruibinx/.local/lib/python3.9/site-packages/pennylane/ops/qubit/non_parametric_ops.py:2095: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(
